In [1]:
data_var = '2024-01-18'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
10148,2024-01-18,Brasil Nbb,19:30,Bauru,Fortaleza B.C.,1.43,2.68,153.5,1.80,1.90,-7.5,1.95,1.74,nNDprYOC,0.699301,0.373134,0.555556,0.526316,0.072435,0.8,0.6,NaN,NaN,176.102,40.747540,0.231386,193.980,136.825595,0.705359,116.532,293.002,141.96,429.00,0.0,0.0,0.0,0.0,0.430114,0.038222,0.080484,1.43,0.286,1.503497,0.000000,0.0,0.000000,-5.00,-1.000,-1.680000,0.365030,0.4,0.034970
10149,2024-01-18,Brasil Nbb,20:00,Botafogo,Caxias do Sul,1.62,2.20,158.5,1.87,1.79,-4.5,2.08,1.67,bcHlshvJ,0.617284,0.454545,0.534759,0.558659,0.071829,0.6,0.6,NaN,NaN,264.716,78.575836,0.296831,282.180,168.656010,0.597689,215.182,257.048,187.68,164.25,0.0,0.0,0.0,0.0,0.214724,0.030912,0.154621,2.35,0.470,1.319149,0.000000,0.0,0.000000,-1.14,-0.228,-5.263158,0.000000,0.0,0.000000
10150,2024-01-18,Brasil Nbb,20:00,Mogi,Corinthians Paulista,2.45,1.51,156.5,1.83,1.87,2.5,2.04,1.66,02eX5yHJ,0.408163,0.662252,0.546448,0.534759,0.070415,0.6,0.4,NaN,NaN,281.178,99.408975,0.353545,165.538,46.354987,0.280026,283.508,161.302,149.25,193.20,0.0,0.0,0.0,0.0,0.335697,0.015289,0.145244,-3.07,-0.614,-2.361564,0.000000,0.0,0.000000,-2.02,-0.404,-1.262376,0.000000,0.0,0.000000
10151,2024-01-18,Eua Nba,00:00,Portland Trail Blazers,Brooklyn Nets,3.07,1.41,220.5,1.82,2.02,5.5,2.08,1.78,ENGJqSon,0.325733,0.709220,0.549451,0.495050,0.034953,0.6,0.0,NaN,NaN,583.822,79.244154,0.135733,255.156,89.594344,0.351136,350.952,276.676,662.36,240.72,1.0,0.0,0.0,0.0,0.524017,0.073657,0.109913,0.61,0.122,16.967213,0.426449,0.3,-0.126449,-5.00,-1.000,-0.410000,0.447988,0.3,-0.147988
10152,2024-01-18,Eua Nba,21:30,Toronto Raptors,Chicago Bulls,2.23,1.77,222.5,1.88,2.05,1.5,2.07,1.85,QspclX4d,0.448430,0.564972,0.531915,0.487805,0.013402,0.4,0.2,NaN,NaN,397.986,148.364514,0.372788,196.184,38.572773,0.196615,244.064,373.478,356.16,235.69,1.0,0.0,0.0,0.0,0.162635,0.061175,0.079369,-3.15,-0.630,-1.952381,0.525745,0.4,-0.125745,1.29,0.258,2.984496,0.326266,0.4,0.073734
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10247,2024-01-18,Eua Ncaa,22:30,Southern Indiana,AR Little Rock,2.38,1.63,149.5,1.80,1.95,2.5,2.00,1.71,dtXQvOJA,0.420168,0.613497,0.555556,0.512821,0.033665,0.0,0.6,NaN,NaN,167.882,50.748028,0.302284,146.756,27.105108,0.184695,0.000,151.006,99.71,156.96,0.0,0.0,0.0,0.0,0.264504,0.056569,0.110545,0.00,0.000,inf,0.000000,0.0,0.000000,-1.10,-0.220,-2.863636,0.000000,0.0,0.000000
10248,2024-01-18,Eua Ncaa,22:30,Southern Utah,Utah Tech,1.58,2.47,154.5,1.80,1.95,-5.5,2.05,1.71,Em8FtapD,0.632911,0.404858,0.555556,0.512821,0.037770,0.8,0.4,NaN,NaN,230.902,78.795572,0.341251,492.738,453.538547,0.920446,195.582,430.498,273.06,324.36,0.0,0.0,0.0,0.0,0.310778,0.056569,0.127881,-1.71,-0.342,-1.695906,0.000000,0.0,0.000000,0.42,0.084,17.500000,0.000000,0.0,0.000000
10249,2024-01-18,Eua Ncaa,22:30,Tennessee State,Tennessee Tech,1.34,3.37,142.5,1.91,1.91,-8.5,2.10,1.67,zRrItpkb,0.746269,0.296736,0.523560,0.523560,0.043005,0.0,0.4,NaN,NaN,154.988,93.270998,0.601795,213.620,115.343365,0.539946,0.000,325.488,90.00,162.06,0.0,0.0,0.0,0.0,0.609523,0.000000,0.161303,0.00,0.000,inf,0.000000,0.0,0.000000,-2.78,-0.556,-4.262590,0.000000,0.0,0.000000
10250,2024-01-18,Eua Ncaa,23:00,Idaho State,Idaho,1.47,2.79,133.5,1.91,1.91,-6.5,2.10,1.67,rP91d8et,0.680272,0.358423,0.523560,0

## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,00:00,Eua Nba,Portland Trail Blazers,Brooklyn Nets,220.5,1.82,0.771,Over
1,21:30,Eua Nba,Toronto Raptors,Chicago Bulls,222.5,1.88,1.000,Over
2,23:00,Eua Nba,Utah Jazz,Oklahoma City Thunder,244.5,1.85,1.000,Over
3,21:30,Eua Nba,New York Knicks,Washington Wizards,233.5,1.85,0.751,Over
